In [1]:
import platform
from subprocess import check_output
import sys

In [9]:
!pip install pesq pystoi

In [10]:
import librosa
import numpy as np
from pesq import pesq
from pystoi import stoi

In [11]:
def calculate_mos(reference_audio_path, test_audio_path, sample_rate=16000):
    # Load the audio files
    ref_audio, ref_sr = librosa.load(reference_audio_path, sr=sample_rate)
    test_audio, test_sr = librosa.load(test_audio_path, sr=sample_rate)
    
    # Resample if necessary
    if ref_sr != sample_rate:
        ref_audio = librosa.resample(ref_audio, orig_sr=ref_sr, target_sr=sample_rate)
    if test_sr != sample_rate:
        test_audio = librosa.resample(test_audio, orig_sr=test_sr, target_sr=sample_rate)
    
    # Ensure both audios are the same length
    min_length = min(len(ref_audio), len(test_audio))
    ref_audio = ref_audio[:min_length]
    test_audio = test_audio[:min_length]
    
    # Calculate PESQ score (wideband mode)
    pesq_score = pesq(sample_rate, ref_audio, test_audio, 'wb')
    
    # Calculate STOI score
    stoi_score = stoi(ref_audio, test_audio, sample_rate)
    
    # Combine metrics to approximate MOS
    mos = (pesq_score + (stoi_score * 5)) / 2  # Normalize STOI to a 1-5 scale
    
    return {
        "PESQ": pesq_score,
        "STOI": stoi_score,
        "Estimated MOS": mos
    }

In [12]:
# Example usage
reference_audio = "/kaggle/input/wavfile1/844424932394604-37-f.wav"
test_audio = "/kaggle/input/wavfile2/844424930841695-70-f.wav"

In [13]:
scores = calculate_mos(reference_audio, test_audio)
print("PESQ:", scores["PESQ"])
print("STOI:", scores["STOI"])
print("Estimated MOS:", scores["Estimated MOS"])


PESQ: 1.0271638631820679
STOI: 0.23395817605739863
Estimated MOS: 1.0984773717345306
